In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)
print(api_key)

1cc89202e555079b554803092d2bff0b


In [2]:
 # List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = list(zip(lats, lngs))

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

614

In [3]:
smaller = cities[0:10]
weather_results = []

print('Beginning Data Retrieval')     
print('-----------------------------')
for i in range(0, 10):
    lat_long = lat_lngs[i]
    lat = lat_long[0]
    long = lat_long[1]
    city = cities[i]
    print(f'Processing Record {i} | {city} at {lat}, {long}')
    time.sleep(2)
    response = requests.get(f'http://api.openweathermap.org/data/2.5/weather?units=imperial&lat={lat}&lon={long}&APPID={api_key}')
    if response.status_code == 200:
        json = response.json()
        temp_max = json['main']['temp_max']
        humidity = json['main']['humidity']
        wind_speed = json['wind']['speed']
        cloudiness = json['clouds']['all']
        latitude = json['coord']['lat']
        longitude = json['coord']['lon']
        country = json['sys'].get('country', 'N/A')
        dt = json['dt']
        weather_results.append({ 'city': city, 'cloudiness': cloudiness, 'country': country, 'date': dt, 'humidity': humidity, 'lat': latitude, 'lng': longitude, 'max_temp': temp_max, 'max_speed': wind_speed})
    else:
        print('City not found. Skipping...')

print('-----------------------------')
print('Data Retrieval Complete')     
print('-----------------------------')
                            

Beginning Data Retrieval
-----------------------------
Processing Record 0 | tumannyy at 78.31059023002894, 36.84782230515316
Processing Record 1 | tasiilaq at 76.50222831678849, -38.79785002606539
Processing Record 2 | dovers at 12.787354704422413, -60.66408859646718
Processing Record 3 | leningradskiy at 68.13723757040566, 175.6318688065457
Processing Record 4 | ushuaia at -74.8857936462972, -60.33798200961394
Processing Record 5 | port alfred at -68.75236896805164, 38.2464540624741
Processing Record 6 | hilo at 14.796788689897014, -149.98635160597746
Processing Record 7 | chokurdakh at 88.13383232190318, 152.30637580512422
Processing Record 8 | cerfontaine at 50.12746414728329, 4.468308934333521
Processing Record 9 | margate at -37.92897258482259, 38.753395543236394
-----------------------------
Data Retrieval Complete
-----------------------------


In [4]:
with open(output_data_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat', 'Lng', 'Max Temp', 'Wind Speed'])

    # Write the second row
    for weather_dictionary in weather_results:
        csvwriter.writerow([weather_dictionary['city'], weather_dictionary['cloudiness'], weather_dictionary['country'], weather_dictionary['date'], weather_dictionary['humidity'], weather_dictionary['lat'], weather_dictionary['lng'], weather_dictionary['max_temp'], weather_dictionary['max_speed']])


In [6]:
weather_df = pd.DataFrame(weather_results)
weather_df.head()

,city,cloudiness,country,date,humidity,lat,lng,max_speed,max_temp
0,tumannyy,49,N/A,1570476251,83,78.31,36.85,19.93,33.17
1,tasiilaq,65,N/A,1570476253,96,76.50,-38.80,9.17,-4.09
2,dovers,75,VC,1570476256,94,12.79,-60.67,5.82,78.80
3,leningradskiy,99,N/A,1570476258,94,68.14,175.63,7.78,17.15
4,ushuaia,100,N/A,1570476260,95,-74.89,-60.34,8.52,-2.11
